## Timing with the timeit Module ##

As we saw in Notebook 6.1 we can use the magic functions to learn the execution time of an entire cell of code. Let us quickly remind ourselves of how the `%%timeit` magic function is used. The function `f` used in the example below is not important - it is designed to just take some non-negligible time, we are simply demonstrating use of the `%%timeit` command.

In [1]:
%%timeit

def f(x):
    for i in range (1,x):
        x += x % 991
    return x

f(1000)

98.6 μs ± 5.42 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Using timeit

The `%%timeit` magic function is convenient, and very simple to use if we just want to gauge how long it takes to run a single function with a specific input. However, it may be more interesting to time our function across a range of inputs, and there is no straightforward way to do this with `%%timeit`. However, Python does provide us with the `timeit` module, which provides us with some useful funtions for profiling our code.

As usual, we must import this module, and then we can use the functions it provides.

In [2]:
import timeit

We will use the same function as we used above, which serves no specific purpose other than to take a reasonable amount of time to execute. We only need to define it once to use it in all of our examples in this notebook.

In [3]:
def f(x):
    for i in range (1,x):
        x += x % 991
    return x

Now let us use the `timeit()` function provided by the timeit module to see how long it takes to execute our function with an input of 10.

In [4]:
timeit.timeit('f(10)', globals=globals())

0.9738112000050023

Notice that we had to pass into the timeit function the name of our function and its input as a string. We will not worry about the `globals=globals()` parameter, we should just remember that we need it. Technically this causes the timing code to be executed within the current global namespace. 

The data returned is the wall clock time in seconds that was required to run our function 1,000,000 times. This is because the timeit function has a default argument called `number` which takes the value 1,000,000 unless we override it. 
One key reason for this is that the timeit function measures wall clock time and the data can sometimes be skewed by other processes running on your machine. Running the function a large number of times allows us to calculate the average runtime, which can help account for inconsistencies in the data.

Now let us try overriding the `number` argument with the value 100,000. This is 10 times smaller than the default value, so the run time should be around 10 times faster.

In [8]:
timeit.timeit('f(10)', globals=globals(), number=100000)

0.10392109997337684

We should remember to divide the runtime returned by the value of `number` if we want to learn how long a single execution of our function takes on average.

From a statistical viewpoint, you might be interested in more of the underlying data associated with multiple executions of your function. One way to achieve this is to use the `repeat()` function instead of the `timeit()` function. We can then provide a `repeat` parameter as follows:  

In [9]:
timeit.repeat('f(10)', globals=globals(), number=100000, repeat=10)

[0.13559679995523766,
 0.11800249997759238,
 0.10899290000088513,
 0.09678369999164715,
 0.09013379999669269,
 0.09638530004303902,
 0.08898800000315532,
 0.08877590001793578,
 0.09342470002593473,
 0.09838649997254834]

If the timings are consistent, this should hopefully imply that no other processes are interfering with the execution time of your function. This is not a fool-proof solution, there are alternative methods for timing CPU time which would be more accurate, however this is generally good enough to understand how well our code is running.

According to the Python documentation the most interesting value of those returned is the minimum. This provides a bound on how fast your machine can run the function that you have provided. Calculating the mean and standard deviation may be misleading if other processes are interfering with your timing accuracy.

### Timing functions with variable input sizes ###

So far we have only considered examples with a fixed input to our function, the value 10. It is obviously much more interesting if we can adjust the input value and store any relevant data relating to that specific input. Let us consider the example below where we use a for loop to determine the value of the variable `x` and we provide that as an input to our function for timing. 

In [10]:
#Let the input size start at 100, and step up in increments of 100.
for x in range (100,1100,100):
    y = timeit.timeit('f(x)', globals=globals(), number=1000)
    print("x =",x, ", y =",y)

x = 100 , y = 0.011108100006822497
x = 200 , y = 0.026033000031020492
x = 300 , y = 0.04036609997274354
x = 400 , y = 0.037264699989464134
x = 500 , y = 0.050804800004698336
x = 600 , y = 0.09834520000731573
x = 700 , y = 0.12782049999805167
x = 800 , y = 0.12137050001183525
x = 900 , y = 0.11096020002150908
x = 1000 , y = 0.11071639996953309


If we want to interpret this data we must pay attention to the fact that we ran our function 1000 times for each input value. Therefore if we want to know how long a single execution of our function takes for a given input we must divide the runtime by the number of repeated experiments, as we can see in the code below: 

In [11]:
z = 1000
for x in range (100,1100,100):
    y = timeit.timeit('f(x)', globals=globals(), number=z)
    print("x =",x, ", y =",y, ", Average =", y/z)


x = 100 , y = 0.018298999988473952 , Average = 1.829899998847395e-05
x = 200 , y = 0.02279299998190254 , Average = 2.2792999981902538e-05
x = 300 , y = 0.03319059999193996 , Average = 3.319059999193996e-05
x = 400 , y = 0.036390399967785925 , Average = 3.6390399967785924e-05
x = 500 , y = 0.051433100015856326 , Average = 5.143310001585633e-05
x = 600 , y = 0.06915809999918565 , Average = 6.915809999918565e-05
x = 700 , y = 0.07739809999475256 , Average = 7.739809999475256e-05
x = 800 , y = 0.1104660999844782 , Average = 0.00011046609998447821
x = 900 , y = 0.0954560999525711 , Average = 9.54560999525711e-05
x = 1000 , y = 0.1158047000062652 , Average = 0.0001158047000062652


Should we want to present this data nicely we could store the output in either a Python list or dictionary or even a NumPy ndarray and, as we will in the next topic, we could then use this data with matplotlib to draw some useful graphs. 

### A note on Complexity ###

This content may not make much sense until you have seen some basic complexity theory. This will be covered in both Computational Thinking and Algorithms and Data Structures in the second half of first term. You may want to leave this final section for now if you are unaware of Big-O notation, and revisit this later in the term. You can however move on and complete Notebooks 6.3, 6.4 and 6.5 without this content.

At this point it is also possible to use this data as a proxy for the complexity of the implementation of an algorithm. You can time your implementation across a range of input sizes and investigate how the runtime changes with the input size. For example, if your runtime data shows a clear doubling when the size of the input doubles it suggests that your implementation has linear complexity, $O(n)$. Similarly if the runtime data shows a quadrupling pattern when the input size is doubled your implementation likely has quadratic complexity, $O(n^2)$. If you are aware of the complexity of the algorithm that you have tried to implement, this data can provide a sanity check on whether you have implemented your algorithm reasonably efficiently.

If we look at our data, we should see this rough doubling of the runtime when we double the input size, suggesting our function has linear complexity, so is $O(n)$. This seems to make sense, we have a single for loop `for i in range (1,x)` where the range scales linearly with the input `x`, and the code inside the loop is a single operation with runtime that appears independent of the value of `x`.

We must be careful however, these runtimes are very small and therefore prone to error. We would want to use larger inputs with much larger runtimes to have greater confidence in identifying patterns in the data. The example below will take a couple of minutes to execute.

In [12]:
z = 20
for x in range (1000000,11000000,1000000):
    y = timeit.timeit('f(x)', globals=globals(), number=z)
    print("x =",x, ", y =",y, ", Average =", y/z)

x = 1000000 , y = 2.435339500021655 , Average = 0.12176697500108276
x = 2000000 , y = 4.021304100053385 , Average = 0.20106520500266925
x = 3000000 , y = 6.344510700029787 , Average = 0.3172255350014893
x = 4000000 , y = 9.692566199984867 , Average = 0.48462830999924333
x = 5000000 , y = 11.93460400000913 , Average = 0.5967302000004565
x = 6000000 , y = 14.144437499984633 , Average = 0.7072218749992316


KeyboardInterrupt: 

Now we see that the linear pattern is not quite as clear. The likely reason is that our assumption the line of code inside the for loop `x += x % 991` is independent of `x` may not be quite true. In fact, most modulo algorithms have complexity $O((log(n))^2)$, so the time required to calculate the modulo will grow with larger values of `x`. This suggests our algorithm probably has complexity of $O(n*(log(n))^2)$, so scales slightly worse than linear. 

This may be even more obvious if we use a while loop instead, and double the input size on every iteration.

In [13]:
x = 200000
z = 20

while (x<20000000):
    y = timeit.timeit('f(x)', globals=globals(), number=z)
    print("x =",x, ", y =",y, ", Average =", y/z)
    x = 2*x

x = 200000 , y = 0.4435360999777913 , Average = 0.022176804998889566
x = 400000 , y = 0.8825682000024244 , Average = 0.04412841000012122
x = 800000 , y = 2.547386699996423 , Average = 0.12736933499982114
x = 1600000 , y = 3.382051100023091 , Average = 0.16910255500115454


KeyboardInterrupt: 

Just remember that generally we would not use timing data to make a guess at the complexity of a particular algorithm. However, it can prove a useful sanity check to ensure that your implementation of an algorithm scales as you would expect.